**Вежба за Глава 4 - Ограничени Болцманови машини**

Кодот е изваден и адаптиран од  https://raw.githubusercontent.com/IBM/dl-learning-path-assets/main/unsupervised-deeplearning/notebooks/CollabortiveFilteringUsingRBM.ipynb.

Ги импортираме потребните библиотеки.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

**Податочно множество**

Ќе илустрираме пример за користење на Ограничени Болцманови машини за колаборативно филтрирање со цел да се развие систем за препораки на филмови.

Податочното множество може да се превземе со следниве наредби:

In [ ]:
!wget -c https://raw.githubusercontent.com/fawazsiddiqi/recommendation-system-with-a-Restricted-Boltzmann-Machine-using-tensorflow/master/data/ml-1m.zip -O moviedataset.zip
!unzip -o moviedataset.zip

--2021-12-01 10:36:05--  https://raw.githubusercontent.com/fawazsiddiqi/recommendation-system-with-a-Restricted-Boltzmann-Machine-using-tensorflow/master/data/ml-1m.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>]   5.64M  --.-KB/s    in 0.03s   

2021-12-01 10:36:06 (196 MB/s) - ‘moviedataset.zip’ saved [5917549/5917549]

Archive:  moviedataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


Потоа го вчитуваме податочното множество при што наведуваме дека како сепаратор треба да се користи ‘::’. За параметарот header проследуваме вредност None бидејќи во фајловите во податочното множество нема заглавје (header) на колоните.

In [ ]:
movies_df = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python')
movies_df.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python')
ratings_df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Следно, им доделуваме имиња на атрибутите.

In [ ]:
movies_df.columns = ['MovieID', 'Title', 'Genres']
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Потоа правиме нормализација на вредностите на оцените така што ги доведуваме да бидат вредности во интервалот [0,1].

In [ ]:
user_rating_df = ratings_df.pivot(index='UserID', columns='MovieID', values='Rating')
norm_user_rating_df = user_rating_df.fillna(0) / 5.0
trX = norm_user_rating_df.values
trX[0:5]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**Поставување на параметрите и креирање на мрежата**

Бројот на неврони во скриениот слој го поставуваме на 20.

In [ ]:
visibleUnits =  len(user_rating_df.columns) #Број на влезни неврони 
hiddenUnits = 20 #Број на скриени неврони

vb = tf.Variable(tf.zeros([visibleUnits]), tf.float32)
hb = tf.Variable(tf.zeros([hiddenUnits]), tf.float32)
W = tf.Variable(tf.zeros([visibleUnits, hiddenUnits]), tf.float32)

Следно, ги креираме влезниот (видливиот) и скриениот слој и специфицираме која активациска функција ќе се користи.

Дефинираме функција hidden_layer за креирање на скриениот слој.

In [ ]:
def hidden_layer(v0_state, W, hb):
    h0_prob = tf.nn.sigmoid(tf.matmul([v0_state], W) + hb)  #probabilities of the hidden units
    h0_state = tf.nn.relu(tf.sign(h0_prob - tf.random.uniform(tf.shape(h0_prob)))) #sample_h_given_X
    return h0_state

Дефинираме функција reconstructed_output за реконструкција на сигналот.

In [ ]:
def reconstructed_output(h0_state, W, vb):
    v1_prob = tf.nn.sigmoid(tf.matmul(h0_state, tf.transpose(W)) + vb) 
    v1_state = tf.nn.relu(tf.sign(v1_prob - tf.random.uniform(tf.shape(v1_prob)))) #sample_v_given_h
    return v1_state[0]

Потоа ја креираме невронската мрежа.

In [ ]:
v0 = tf.zeros([visibleUnits], tf.float32)

h0 = hidden_layer(v0, W, hb)

v1 = reconstructed_output(h0, W, vb)

Со следниве наредби може да го погледнеме бројот на неврони во секој слој.

In [ ]:
print("hidden state shape: ", h0.shape)
print("v0 state shape:  ", v0.shape)
print("v1 state shape:  ", v1.shape)

hidden state shape:  (1, 20)
v0 state shape:   (3706,)
v1 state shape:   (3706,)


Дефинираме функција error за процена на грешката.

In [ ]:
def error(v0_state, v1_state):
    return tf.reduce_mean(tf.square(v0_state - v1_state))

err = tf.reduce_mean(tf.square(v0 - v1))

**Тренирање на мрежата**

Мрежата нека се тренира 5 епохи. Во примерот се користат batch-ови со големина 500 со што се добиваат 12 batch-ови.

In [ ]:
epochs = 5
batchsize = 500
errors = []
weights = []
K=1
alpha = 0.1

train_ds = tf.data.Dataset.from_tensor_slices((np.float32(trX))).batch(batchsize)

v0_state=v0
for epoch in range(epochs):
    batch_number = 0
    for batch_x in train_ds:

        for i_sample in range(len(batch_x)):           
            for k in range(K):
                v0_state = batch_x[i_sample]
                h0_state = hidden_layer(v0_state, W, hb)
                v1_state = reconstructed_output(h0_state, W, vb)
                h1_state = hidden_layer(v1_state, W, hb)

                delta_W = tf.matmul(tf.transpose([v0_state]), h0_state) - tf.matmul(tf.transpose([v1_state]), h1_state)
                W = W + alpha * delta_W

                vb = vb + alpha * tf.reduce_mean(v0_state - v1_state, 0)
                hb = hb + alpha * tf.reduce_mean(h0_state - h1_state, 0) 

                v0_state = v1_state

            if i_sample == len(batch_x)-1:
                err = error(batch_x[i_sample], v1_state)
                errors.append(err)
                weights.append(W)
        batch_number += 1

**Препорачување на филмови**

Следно, може да се препорачуваат филмови за даден корисник (се специфицира неговото ID). 



In [ ]:
rec_user_id = 100

На влез се даваат преференците на корисникот (филмови кои ги гледал тој корисник), со што со мрежата треба да се реконструира влезот односно да се процени кои филмови би му се допаднале на корисникот врз база на неговите преференци.

In [ ]:
inputUser = trX[rec_user_id-1].reshape(1, -1)

inputUser = tf.convert_to_tensor(trX[rec_user_id-1],"float32")
v0 = inputUser

Врз база на влезниот вектор (преференците на корисникот) правиме реконструкција на влезот.

In [ ]:
hh0 = tf.nn.sigmoid(tf.matmul([v0], W) + hb)

vv1 = tf.nn.sigmoid(tf.matmul(hh0, tf.transpose(W)) + vb)

rec = vv1

tf.maximum(rec,1)

<tf.Tensor: shape=(1, 3706), dtype=float32, numpy=array([[1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

Реконструираниот вектор може да го погледнеме со:

In [ ]:
for i in vv1:
    print(i)

tf.Tensor(
[3.4984052e-03 2.8870404e-03 8.8551641e-04 ... 3.8683414e-04 5.0342554e-05
 2.9301643e-04], shape=(3706,), dtype=float32)


Потоа може да ги вратиме првите N (во случајов 10) највисоко рангирани филмови за тој корисник.

In [ ]:
scored_movies_df_rec = movies_df[movies_df['MovieID'].isin(user_rating_df.columns)]
scored_movies_df_rec = scored_movies_df_rec.assign(RecommendationScore = rec[0])
scored_movies_df_rec.sort_values(["RecommendationScore"], ascending=False).head(10)

,MovieID,Title,Genres,RecommendationScore
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,0.907549
453,457,"Fugitive, The (1993)",Action|Thriller,0.789628
1182,1200,Aliens (1986),Action|Sci-Fi|Thriller|War,0.731198
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,0.707811
847,858,"Godfather, The (1972)",Action|Crime|Drama,0.697240
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War,0.686962
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller,0.685480
1023,1036,Die Hard (1988),Action|Thriller,0.681653
1959,2028,Saving Private Ryan (1998),Action|Drama|War,0.667807
108,110,Braveheart (1995),Action|Drama|War,0.603830


Треба да одредиме кои филмови сеуште ги нема гледано тој корисник.
За таа цел прво одредуваме кои филмови ги има гледано корисникот.

In [ ]:
movies_df_rec = ratings_df[ratings_df['UserID'] == rec_user_id]
movies_df_rec.head()

,UserID,MovieID,Rating,Timestamp
12900,100,648,2,977594265
12901,100,800,5,977593915
12902,100,3948,3,977593808
12903,100,1408,3,977594113
12904,100,1196,4,977593950


Потоа се спојуваат movies_df и ratings_df според MovieID.

Со how='outer' специфицираме дека треба да се направи надворешно спојување (outer join).

In [ ]:
merged_df_rec = scored_movies_df_rec.merge(movies_df_rec, on='MovieID', how='outer')

Потоа ги враќаме првите N највисоко рангирани филмови.

In [ ]:
merged_df_rec.sort_values(["RecommendationScore"], ascending=False).head(10)

,MovieID,Title,Genres,RecommendationScore,UserID,Rating,Timestamp
1120,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,0.907549,100.0,4.0,977593637.0
443,457,"Fugitive, The (1993)",Action|Thriller,0.789628,NaN,NaN,NaN
1110,1200,Aliens (1986),Action|Sci-Fi|Thriller|War,0.731198,100.0,3.0,977594021.0
253,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,0.707811,100.0,4.0,977593595.0
802,858,"Godfather, The (1972)",Action|Crime|Drama,0.697240,100.0,4.0,977593950.0
737,780,Independence Day (ID4) (1996),Action|Sci-Fi|War,0.686962,100.0,1.0,977594378.0
1124,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller,0.685480,NaN,NaN,NaN
971,1036,Die Hard (1988),Action|Thriller,0.681653,NaN,NaN,NaN
1848,2028,Saving Private Ryan (1998),Action|Drama|War,0.667807,100.0,4.0,977593988.0
106,110,Braveheart (1995),Action|Drama|War,0.603830,100.0,3.0,977594044.0


Филмовите кои ќе му ги препорачаме на корисникот се филмовите за кои е добиена висока вредност за RecommendationScore, а кои сеуште ги нема гледано тој корисник.